# Digit Recognizer

* This notebook contains the code for the Digit Recognizer Competition.
* in this notebook, I tried out architectures using Conv2D, Dense, Dropout, MaxPool2D and BatchNormalization.
* I also used various optimizers i.e. RMSProp, Adam, SGD and different callbacks like Reduce Learning Rate on Plateau and Early Stop.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, BatchNormalization

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import regularizers, optimizers

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.datasets import mnist

# Ignore the warnings
import warnings

# suppress display of warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
# Extract label
y = train['label']

# Extract features
features = train.drop('label', axis=1)

In [ ]:
print(train.shape, test.shape, y.shape)

In [ ]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = x_train.reshape(60000, -1)
# x_test = x_test.reshape(10000, -1)
# train = np.concatenate([train, x_train, x_test], axis = 0)
# y = np.concatenate([y, y_train, y_test], axis = 0)
# print(train.shape, y.shape)

In [ ]:
# train = tf.reshape(train, (-1, 28, 28, 1))
# test = tf.reshape(test, (-1, 28, 28, 1))
# print(train.shape, test.shape)

In [ ]:
# Train images
X = np.array(features)
X_train = X.reshape(X.shape[0], 28, 28)

# Test images
X_test = np.array(test)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
print("X_train shape:", X_train.shape)
print("Images in X_train:", X_train.shape[0])
print("Images in X_test:", X_test.shape[0])
print("Max value in X_train:", X_train.max())
print("Min value in X_train:", X_train.min())

In [ ]:
y = to_categorical(y, num_classes=10)      
print("Shape of y_train:", y.shape)      
print("One value of y_train:", y[0])      

In [ ]:
# model = Sequential() 
# model.add(Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(28, 28, 1))) 
# model.add(Conv2D(filters=32, kernel_size=3, activation="relu")) 
# model.add(Flatten()) 
# model.add(Dense(128, activation="relu")) 
# model.add(Dense(10, activation="softmax")) 

In [ ]:
# # Compile the model
# model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

# # Fit the model
# model.fit( X_train, y, batch_size=32, epochs=20, validation_split = 0.3)

In [ ]:
# Initialize the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 32, kernel_size=3, activation="relu"))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())
model.add(Dropout(rate = 0.2))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(10, activation = "softmax"))

In [ ]:
# Optimizer
sgd = optimizers.SGD(lr = 2e-2, decay = 1e-6, momentum = 0.9)
  
# Compile the model 
model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer=sgd)

In [ ]:
# Adding callbacks
# es = EarlyStopping(monitor='val_loss', mode = 'min', patience=10, min_delta=1E-4, restore_best_weights=True)
rlrp = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.01, patience = 10, min_delta = 1E-4)

callbacks = [rlrp]
# Fit the model
history = model.fit(x = X_train, y = y, batch_size = 16, epochs = 100, validation_split = 0.3, callbacks=[callbacks]) 

In [ ]:
# Predict on Test set
preds = np.argmax(model.predict(X_test), axis=1)  

In [ ]:
submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv') 

In [ ]:
submission['Label'] = preds
submission.to_csv('submission_1.csv',index=False) 